# Supervised Fine-Tuning with SFTTrainer

This notebook demonstrates how to fine-tune the `HuggingFaceTB/SmolLM2-135M` model using the `SFTTrainer` from the `trl` library. The notebook cells run and will finetune the model. You can select your difficulty by trying out different datasets.

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>Exercise: Fine-Tuning SmolLM2 with SFTTrainer</h2>
    <p>Take a dataset from the Hugging Face hub and finetune a model on it. </p> 
    <p><b>Difficulty Levels</b></p>
    <p>🐢 Use the `HuggingFaceTB/smoltalk` dataset</p>
    <p>🐕 Try out the `bigcode/the-stack-smol` dataset and finetune a code generation model on a specific subset `data/python`.</p>
    <p>🦁 Select a dataset that relates to a real world use case your interested in</p>
</div>

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Install the requirements in Google Colab
# !pip install transformers datasets trl huggingface_hub

# Authenticate to Hugging Face

from huggingface_hub import login

login()

# for convenience you can create an environment variable containing your hub token as HF_TOKEN

In [3]:
# Import necessary libraries
from typing import Dict

import torch
from datasets import DatasetDict, load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer
from trl import SFTConfig, SFTTrainer, setup_chat_format

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

# Load the model and tokenizer
model_name = "HuggingFaceTB/SmolLM2-135M"
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name
).to(device)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)

# Set up the chat format
model, tokenizer = setup_chat_format(model=model, tokenizer=tokenizer)
model.config.eos_token_id = 3 

# Set our name for the finetune to be saved &/ uploaded to
finetune_name = "SmolLM2-FT-Haiku"
finetune_tags = ["smol-course", "module_1"]

# Generate with the base model

Here we will try out the base model which does not have a chat template. 

In [4]:
# Let's test the base model before training
prompt = "Write a haiku about programming"

# Format with template
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

# Generate response
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=100)
print("Before training:")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Before training:
user
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a


## Dataset Preparation

We will load a sample dataset and format it for training. The dataset should be structured with input-output pairs, where each input is a prompt and the output is the expected response from the model.

**TRL will format input messages based on the model's chat templates.** They need to be represented as a list of dictionaries with the keys: `role` and `content`,.

In [5]:
ds = load_dataset(path="davanstrien/haiku_dpo", name="default")
split_ds = ds["train"].train_test_split(test_size=0.2, seed=42)
ds = DatasetDict({"train": split_ds["train"], "test": split_ds["test"]})

display(ds)
display(ds["train"][0])

DatasetDict({
    train: Dataset({
        features: ['question', 'generation_model', 'generation_prompt', 'generations', 'scores', 'chosen', 'chosen_score', 'rejected', 'rejected_score', 'tie', 'difference_in_score', 'system'],
        num_rows: 3298
    })
    test: Dataset({
        features: ['question', 'generation_model', 'generation_prompt', 'generations', 'scores', 'chosen', 'chosen_score', 'rejected', 'rejected_score', 'tie', 'difference_in_score', 'system'],
        num_rows: 825
    })
})

{'question': 'Write a haiku about the melting glacier and its connection to climate change.',
 'generation_model': ['TheBloke/OpenHermes-2.5-Mistral-7B-AWQ',
  'TheBloke/OpenHermes-2.5-Mistral-7B-AWQ',
  'TheBloke/OpenHermes-2.5-Mistral-7B-AWQ',
  'TheBloke/OpenHermes-2.5-Mistral-7B-AWQ',
  'TheBloke/OpenHermes-2.5-Mistral-7B-AWQ',
  'TheBloke/OpenHermes-2.5-Mistral-7B-AWQ',
  'TheBloke/OpenHermes-2.5-Mistral-7B-AWQ',
  'TheBloke/OpenHermes-2.5-Mistral-7B-AWQ',
  'TheBloke/OpenHermes-2.5-Mistral-7B-AWQ',
  'TheBloke/OpenHermes-2.5-Mistral-7B-AWQ',
  'TheBloke/OpenHermes-2.5-Mistral-7B-AWQ',
  'TheBloke/OpenHermes-2.5-Mistral-7B-AWQ',
  'TheBloke/OpenHermes-2.5-Mistral-7B-AWQ',
  'TheBloke/OpenHermes-2.5-Mistral-7B-AWQ',
  'TheBloke/OpenHermes-2.5-Mistral-7B-AWQ',
  'TheBloke/OpenHermes-2.5-Mistral-7B-AWQ',
  'TheBloke/OpenHermes-2.5-Mistral-7B-AWQ',
  'TheBloke/OpenHermes-2.5-Mistral-7B-AWQ',
  'TheBloke/OpenHermes-2.5-Mistral-7B-AWQ',
  'TheBloke/OpenHermes-2.5-Mistral-7B-AWQ'],
 'gen

In [6]:
def process_sample(sample: Dict) -> Dict:
    messages = [
        {"role": "system", "content": sample["system"]},
        {"role": "user", "content": sample["question"]},
        {"role": "assistant", "content": sample["chosen"]},
    ]

    return {
        "messages": tokenizer.decode(
            token_ids=tokenizer.apply_chat_template(
                conversation=messages, tokenize=True, add_generation_prompt=False
            )
        )
    }


ds = ds.map(
    process_sample,
    remove_columns=ds["train"].column_names,
    desc="Processing dataset",
)
display(ds)
display(ds["train"][0])

DatasetDict({
    train: Dataset({
        features: ['messages'],
        num_rows: 3298
    })
    test: Dataset({
        features: ['messages'],
        num_rows: 825
    })
})

{'messages': "<|im_start|>system\nYou are a poet specialising in creating Haiku. \nYour haiku consist of three lines, with five syllables in the first line, seven in the second, and five in the third.\nBeyond being technically correct, your haiku should also be beautiful and meaningful<|im_end|>\n<|im_start|>user\nWrite a haiku about the melting glacier and its connection to climate change.<|im_end|>\n<|im_start|>assistant\nMelting ice reveals,\nClimate's wrath, no place to hide,\nMother Earth laments.<|im_end|>\n"}

## Configuring the SFTTrainer

The `SFTTrainer` is configured with various parameters that control the training process. These include the number of training steps, batch size, learning rate, and evaluation strategy. Adjust these parameters based on your specific requirements and computational resources.

In [7]:
# Configure the SFTTrainer
sft_config = SFTConfig(
    dataset_text_field="messages",  # The field in the dataset containing the text
    output_dir="./sft_output",
    max_steps=512,  # Adjust based on dataset size and desired training duration
    per_device_train_batch_size=4,  # Set according to your GPU memory capacity
    learning_rate=5e-5,  # Common starting point for fine-tuning
    logging_steps=8,  # Frequency of logging training metrics
    save_steps=128,  # Frequency of saving model checkpoints
    eval_strategy="steps",  # Evaluate the model at regular intervals
    eval_steps=32,  # Frequency of evaluation
    use_mps_device=(
        True if device == "mps" else False
    ),  # Use MPS for mixed precision training
    hub_model_id=finetune_name,  # Set a unique name for your model
)

# Initialize the SFTTrainer
trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=ds["train"],
    tokenizer=tokenizer,
    eval_dataset=ds["test"],
)

/home/uziel/Development/smol-course/.venv/lib/python3.11/site-packages/trl/trainer/sft_trainer.py:309: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/825 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


## Training the Model

With the trainer configured, we can now proceed to train the model. The training process will involve iterating over the dataset, computing the loss, and updating the model's parameters to minimize this loss.

In [8]:
# Train the model
trainer.train()

# Save the model
trainer.save_model(f"./{finetune_name}")

Step,Training Loss,Validation Loss
32,0.799200,0.784710
64,0.722700,0.710662
96,0.676400,0.686295
128,0.691400,0.665852
160,0.668800,0.654378
192,0.638700,0.644708
224,0.601200,0.634113
256,0.609300,0.625885
288,0.611300,0.620998
320,0.621300,0.614585


In [9]:
# trainer.push_to_hub(tags=finetune_tags)

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>Bonus Exercise: Generate with fine-tuned model</h2>
    <p>🐕 Use the fine-tuned to model generate a response, just like with the base example..</p>
</div>

In [12]:
# Let's test the base model before training
prompt = "Write a haiku about the blue sky"

# Format with template
messages = [
    {
        "role": "system",
        "content": (
            "You are a poet specialising in creating Haiku. \n"
            "Your haiku consist of three lines, with five syllables in the first line, "
            "seven in the second, and five in the third.\n"
            "Beyond being technically correct, your haiku should also be "
            "beautiful and meaningful"
        ),
    },
    {"role": "user", "content": prompt},
]
formatted_prompt = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)

# Generate response
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=100)
print("After training:")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

After training:
system
You are a poet specialising in creating Haiku. 
Your haiku consist of three lines, with five syllables in the first line, seven in the second, and five in the third.
Beyond being technically correct, your haiku should also be beautiful and meaningful
user
Write a haiku about the blue sky
assistant
In the blue sky,
A gentle breeze whispers peace,
Nature's lullaby.
Nature's lullaby.
Peaceful, serene,
Nature's lullaby.
Nature's lullaby.
Peaceful, serene,
Nature's lullaby.
Nature's lullaby.
Peaceful, serene,
Nature's lullaby.
Nature's lullaby.
Peaceful, serene,
Nature's lullaby.



## 💐 You're done!

This notebook provided a step-by-step guide to fine-tuning the `HuggingFaceTB/SmolLM2-135M` model using the `SFTTrainer`. By following these steps, you can adapt the model to perform specific tasks more effectively. If you want to carry on working on this course, here are steps you could try out:

- Try this notebook on a harder difficulty
- Review a colleagues PR
- Improve the course material via an Issue or PR.